In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import string
import lxml
import re
import spacy
from nltk.corpus import cmudict
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import ToktokTokenizer
from nltk.stem import WordNetLemmatizer 
import os

In [2]:
df=pd.read_csv("Output Data Structure.xlsx - Sheet1.csv")
df.URL_ID=df.URL_ID.astype("str")
df.URL_ID.dtypes

for i,url in enumerate(df.URL[:3]):
    a=df.URL_ID[i].replace(".0",".txt")
    r=requests.get(url).text
    soup=BeautifulSoup(r,"lxml")
    os.chdir(r"D:\DS\dspython\asg\Untitled Folder")
    file=open(a,"w", encoding="utf-8")
    for para in soup.find_all("p"):
        body=para.get_text()
        file.write(body)
    file.close()
df["text"]=""

for i,f in enumerate(os.listdir(r"D:\DS\dspython\asg\Untitled Folder")):
       with open(f,"r",encoding="utf-8") as f:
            df["text"][i]=f.read()
         

st=list(set(stopwords.words("english")))
st_l=os.listdir(r"D:\DS\dspython\asg\stopword")
os.chdir(r"D:\DS\dspython\asg\stopword")
for i in st_l:
      with open(i,"r")as f:
        st.extend(f.read().split("\n"))
# nltk.download("omw-1.4")
# nltk.download('wordnet')

tk=ToktokTokenizer()
wn=WordNetLemmatizer()
#-----------------------------------Removing stop words-------------------------
final_sent=[]
df['final_text']=""
for i in range(len(df.text[:15])):
  review=re.sub(r"[^a-z A-Z . ! ?]","",df["text"][i])
  review=tk.tokenize(review)
  review=[wn.lemmatize(word.lower()) for word in review if word not in st]
  final_sent.append(review)
  review=" ".join(review)
  df["final_text"][i]=review  
    


# ----------- Calculate Positive Negative Words-------------------------

os.chdir(r"D:\DS\dspython\asg\master_words")
with open("positive_words.txt","r",)as f:
      pos=f.read().lower().split("\n")
with open("negative_words.txt","r",)as f:
      neg=f.read().lower().split("\n")
positive=0
negative=0
neutral=0
for i in range(len(df["final_text"][:])):
    positive=0
    negative=0
    neutral=0
    for word in tk.tokenize(df["final_text"][i]):
        if word in pos:
             positive+=1
        elif word in neg:
            
            negative+=1
        else:
            neutral+=1
    polarity_score = (positive - negative)/((positive + negative) + 0.000001)
    total_words=positive+negative+neutral
    Subjectivity_Score = (positive + negative)/((total_words)+0.000001)
    df["POSITIVE SCORE"][i]=positive
    df["NEGATIVE SCORE"][i]=negative
#     df["NEUTRAL SCORE"][i]=neutral
    df["POLARITY SCORE"][i]=polarity_score
#     df["TOTAL WORDS"][i]=total_words
    df["SUBJECTIVITY SCORE"][i]=Subjectivity_Score

def break_sentences(text):
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(text)
    return list(doc.sents)
 
    # Returns Number of Words in the text
def word_count(text):
    sentences = break_sentences(text)
    words = 0
    for sentence in sentences:
        words += len([token for token in sentence])
    return words
    
def sentence_count(text):
    sentences = break_sentences(text)
    return len(sentences)
     
    # Returns average sentence length
def avg_sentence_length(text):
    words = word_count(text)
    sentences = sentence_count(text)
    try:
        average_sentence_length = float(words / sentences)
        return average_sentence_length
    except ZeroDivisionError:
        average_sentence_length=0
        return average_sentence_length

def AVG_NUMBER_OF_WORDS_PER_SENTENCE(text):
    sentences_len = sentence_count(text)
    no_of_word=len(tk.tokenize(text))
    try:
        AVG_NUMBER_OF_WORDS_PER_SENTENCE = float(no_of_word / sentences_len)
        return AVG_NUMBER_OF_WORDS_PER_SENTENCE
    except ZeroDivisionError:
        AVG_NUMBER_OF_WORDS_PER_SENTENCE=0
        return AVG_NUMBER_OF_WORDS_PER_SENTENCE
    
    
a=0
for data1 in df["final_text"][:3]:
#     Avg sentence length
    avg_sen_len=avg_sentence_length(data1)
    df["AVG SENTENCE LENGTH"].iloc[a] =avg_sen_len
# Avg number of words per sentence
    AVG_NUMBER_OF_WORDS_PER_SENT=AVG_NUMBER_OF_WORDS_PER_SENTENCE(data1)
    df["AVG NUMBER OF WORDS PER SENTENCE"].iloc[a] = AVG_NUMBER_OF_WORDS_PER_SENT
    a+=1
    

from textstat.textstat import textstatistics
for i in range(len(df["final_text"][:3])):
    k=list()
    count1=list()
    for word in tk.tokenize(df["final_text"][i]):
# Complex words
        b=textstatistics().syllable_count(word)
        k.append(b)

# syllable word 
    
        if word.endswith("ed") or word.endswith("es"):
                pass
        else:
            b=textstatistics().syllable_count(word)
            count1.append(b)
        
    complex_word=0
    for word in k:
        if word > 2:
            complex_word+=1
            
    syllabe_count_per_word=len(count1)
    df["COMPLEX WORD COUNT"][i] = complex_word
    df["SYLLABLE PER WORD"][i] =syllabe_count_per_word
#     print(syllabe_count_per_word)
a=0
for i in range(len(df["final_text"][:3])):
    no_of_word=len(tk.tokenize(df["final_text"][i]))
    Percentage_of_Complex_words=df["COMPLEX WORD COUNT"][i] / no_of_word
    df["PERCENTAGE OF COMPLEX WORDS"][i] = Percentage_of_Complex_words

#     # Fog Index = 0.4 * (Average Sentence Length + Percentage of Complex words)
    fog_index=0.4*(df["AVG SENTENCE LENGTH"][i] + Percentage_of_Complex_words)
    df["FOG INDEX"].iloc[a] = fog_index
    
#     # Word Count
    df["WORD COUNT"].iloc[a] = no_of_word
    a+=1

a=0
for i in range(len(df["final_text"][:3])):
    k=0
    no_of_word=len(tk.tokenize(df["final_text"][i]))
    data1=tk.tokenize(df["final_text"][i])
#     print(no_of_word)
    for word in data1:
        for i in word:
            k+=1
#     Avg_word_len=k /no_of_word
    try:
             Avg_word_len=k /no_of_word
#             return Avg_word_len
    except ZeroDivisionError:
            Avg_word_len=0
#             return Avg_word_len
#     print(Avg_word_len)
    df["AVG WORD LENGTH"].iloc[a] =Avg_word_len
    a+=1

pronounRegex = re.compile(r'\b(I|we|my|ours|(?-i:us))\b',re.I)
pronouns =[len(pronounRegex.findall(df["text"][i])) for i in range(len(df["text"]))]
# print(pronouns)
df["PERSONAL PRONOUNS"]=pronouns



C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\

C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\

C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\

C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"][i]=f.read()
C:\Users\varva\AppData\Local\Temp\

C:\Users\varva\AppData\Local\Temp\ipykernel_5784\904935624.py:185: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["AVG WORD LENGTH"].iloc[a] =Avg_word_len


In [3]:
df.head(3)

,URL_ID,URL,POSITIVE SCORE,NEGATIVE SCORE,POLARITY SCORE,SUBJECTIVITY SCORE,AVG SENTENCE LENGTH,PERCENTAGE OF COMPLEX WORDS,FOG INDEX,AVG NUMBER OF WORDS PER SENTENCE,COMPLEX WORD COUNT,WORD COUNT,SYLLABLE PER WORD,PERSONAL PRONOUNS,AVG WORD LENGTH,text,final_text
0,137,https://insights.blackcoffer.com/what-do-you-t...,17.0,37.0,-0.370370,0.081818,21.343750,0.307576,8.660530,20.625000,203.0,660.0,616.0,2,7.225758,Will COVID19 END Globalization?Globalization:G...,will covid end globalization?globalizationglob...
1,138,https://insights.blackcoffer.com/coronavirus-t...,4.0,3.0,0.142857,0.090909,13.666667,0.220779,5.554978,12.833333,17.0,77.0,73.0,2,6.519481,The UN projects a 20-30% decline in internatio...,the project decline international tourist arri...
2,139,https://insights.blackcoffer.com/industrial-re...,25.0,41.0,-0.242424,0.120000,12.869565,0.261818,5.252553,11.956522,144.0,550.0,488.0,13,6.540000,Perhaps the virtual illusion has become today’...,perhaps virtual illusion today new reality.inq...
